In [1]:
import os

In [4]:
%pwd

'/Users/davidibrahim/Programming/mlops1/Project1'

In [3]:
%cd ..

/Users/davidibrahim/Programming/mlops1/Project1


/Users/davidibrahim/Programming/mlops1/Project1/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:

    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.datascience.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.datascience.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH, 
                 schema_file_path=SCHEMA_FILE_PATH):
        

        self.config = read_yaml(str(config_file_path))
        self.params = read_yaml(str(params_file_path))
        self.schema = read_yaml(str(schema_file_path))

        ## Create directories if they do not exist
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        
        


In [18]:
import os
import urllib.request as request
from src.datascience import logger
from zipfile import ZipFile

In [19]:
## Data ingestion component

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    ## Downloading the zip file from the source URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with headers: \n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists. Skipping download.")



    ## Unzipping the downloaded file
    def extract_zip_file(self):
        """
        zip_file_path: str
        extract the file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")



In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-07-02 15:39:57,943 - datascience - INFO - YAML file config/config.yaml loaded successfully.]
[2025-07-02 15:39:57,944 - datascience - INFO - YAML file params.yaml loaded successfully.]
[2025-07-02 15:39:57,945 - datascience - INFO - YAML file schema.yaml loaded successfully.]
[2025-07-02 15:39:57,945 - datascience - INFO - Directory artifacts created or already exists.]
[2025-07-02 15:39:57,945 - datascience - INFO - Directory artifacts/data_ingestion created or already exists.]
[2025-07-02 15:39:57,946 - datascience - INFO - File artifacts/data_ingestion/data.zip already exists. Skipping download.]
[2025-07-02 15:39:57,949 - datascience - INFO - Extracted zip file to artifacts/data_ingestion]
